In [ ]:
import os

import torch
from torch import nn
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms

import matplotlib.pyplot as plt

In [ ]:
device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using {device}")

In [ ]:
IMAGE_SIZE = 128 #Actual frames = 1440. must be power of 2
IMAGE_CHANNELS = 3
LATENT_VECTOR_SIZE = 100

In [ ]:
def show_examples(model, title=None, big=False):
    """Display examples of generated images

    Args:
        model (nn.Module): The generator model to use
        title (str, optional): The title of the display. Defaults to None.
        big (bool, optional): If false, display 4x1 examples, if true, display 4x3. Defaults to False.
    """
    latent_space_samples = torch.randn((batch_size, LATENT_VECTOR_SIZE))
    generated_samples = model(latent_space_samples)

    generated_samples = generated_samples.detach()
    fig = plt.figure()
    if title:
        fig.text(0.5, 1.0 if big else 0.7, title, ha='center', va='center')
    for plot_idx, dataset_idx  in enumerate(torch.randint(len(generated_samples), (12 if big else 4,))):
        sub = plt.subplot(3 if big else 1, 4, plot_idx+1)
        plt.tight_layout()
        sub.axis('off')
        image = generated_samples[dataset_idx.item()].permute(1,2,0)
        plt.imshow(image *0.5 + 0.5)
    fig.show()

# Preparing training data
Here I have two datasets. The reason is that I initially wanted to generate images of myself, like a good egoistic human would. The problem was that the reference images are very similar, so the generator got good at generating the background and my hair. However, the face was just a blob, destroing the point. Also, when testing on the flower dataset, I noticed that the training went much faster. This is probably a result of better optimizations and transformations when reading the images. I could look closer into optimizing the `VetleDataset`, but the flowers were more interesting to look at anyway. I left the old dataset because it should work given training time.

In [ ]:
class VetleDataset(Dataset):

    def __init__(self, sample_count, image_dir, transform=None):
        """
        Arguments:
            sample_count (int): How many samples
            image_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.sample_count = sample_count
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return self.sample_count

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        frameno = str(idx+1).zfill(4)
        filename = "frame" + frameno + ".png"

        img_name = os.path.join(self.image_dir,
                                filename)
        image = torchvision.io.read_image(img_name).float()
        

        if self.transform:
            image = self.transform(image)

        # Return image with a label. We won't use it, but it seems like tensorflow expects it
        return [image,0]

In [ ]:
vetleTransform = transforms.Compose([ transforms.Resize(IMAGE_SIZE), transforms.Normalize(128,128)])

In [ ]:
vetleDataset = VetleDataset(2905, "assets/frames/", transform=vetleTransform)

In [ ]:
flowerTransform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),  
    transforms.RandomCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) 
])

In [ ]:
flowerDataset = torchvision.datasets.Flowers102(root="assets", download=True, transform=flowerTransform)

In [ ]:
SELECTED_DATASET = flowerDataset

Plot training data:

In [ ]:
fig = plt.figure()
for plot_idx, dataset_idx  in enumerate(torch.randint(len(SELECTED_DATASET), (4,))):
    sub = plt.subplot(1, 4, plot_idx+1)
    plt.tight_layout()
    sub.axis('off')
    image = SELECTED_DATASET[dataset_idx.item()][0].permute(1,2,0)
    plt.imshow(image * 0.5 + 0.5)
fig.show()

Create data loader:

In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(
    SELECTED_DATASET, batch_size=batch_size, shuffle=True
)


# The models


## Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=IMAGE_CHANNELS, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),


            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(in_channels=1024, out_channels=1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid() 
        )

    def forward(self, x):
        output = self.model(x)
        return output.view(x.size(0),1)

In [ ]:
discriminator = Discriminator()

## Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
       
        self.model = nn.Sequential(
            nn.ConvTranspose2d(in_channels=LATENT_VECTOR_SIZE, out_channels=1024, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            
            nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),


            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh(),
        )

    def forward(self, x):
        x = x.view(x.size(0),LATENT_VECTOR_SIZE,1,1)
        return self.model(x)



In [ ]:
generator = Generator()
show_examples(generator, title="Initial", big=True)

# Training


## Hyperparameters

In [ ]:
lr = 0.0002
num_epochs = 300
loss_function = nn.BCELoss()
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5,0.999))
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5,0.999))

Loop

In [ ]:
timeline = {
    "loss_discriminator":[],
    "loss_generator":[],
}
for epoch in range(num_epochs):
    print(epoch, end=" ")
    for n, (real_samples, _) in enumerate(train_loader):
        # Data for training the discriminator
        real_samples_labels = torch.ones((real_samples.shape[0], 1))
        latent_space_samples = torch.randn((batch_size, LATENT_VECTOR_SIZE))
        generated_samples = generator(latent_space_samples)
        generated_samples_labels = torch.zeros((batch_size, 1))
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
        discriminator.zero_grad()
        discriminator_loss = 0
        ## First only real samples
        output_discriminator_real = discriminator(real_samples)
        loss_discriminator_real = loss_function(
            output_discriminator_real, real_samples_labels)
        loss_discriminator_real.backward()
        discriminator_loss += loss_discriminator_real.detach()
        
        ## Then only generated samples
        output_discriminator_gen = discriminator(generated_samples)
        loss_discriminator_gen = loss_function(
            output_discriminator_gen, generated_samples_labels)
        loss_discriminator_gen.backward()
        
        optimizer_discriminator.step()
        discriminator_loss += loss_discriminator_gen.detach()

        # Data for training the generator
        latent_space_samples = torch.randn((batch_size, LATENT_VECTOR_SIZE))
        real_labels = torch.ones((batch_size, 1))

        # Training the generator
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)
        output_discriminator_generated = discriminator(generated_samples)
        loss_generator = loss_function(
            output_discriminator_generated, real_labels
        )
        loss_generator.backward()
        optimizer_generator.step()

        # Show loss
        if n == len(train_loader) - 1:
            timeline["loss_discriminator"].append(discriminator_loss)
            timeline["loss_generator"].append(loss_generator.detach())
            if epoch % 10 == 0 or epoch == num_epochs -1:
                print("")
                print(f"Epoch: {epoch} Loss D.: {discriminator_loss}")
                print(f"Epoch: {epoch} Loss G.: {loss_generator}")
                show_examples(generator, title=f"Epoch {epoch}")
           

# Results
## Example images

In [ ]:
show_examples(generator, title="End Results", big=True)


## A look at lerping
I was concerned that the generator might overfit to the reference images. A perfect way to trick the discriminator is to "learn" one or a few of these images. A way to detect overfitting in GANs is to smoothly lerp between two different latent vectors and look for sudden "jumps" in the output image. A jump like this can signify that the generator has learnt a few examples, and is reciting them based on some value in the input noise.

If we see that the resulting flowers gradually change from one to another, we can guess that the generator actually generates meaningful properties.

In [ ]:
# Sample two random points in latent space
latent_space_endpoints = torch.randn((2, LATENT_VECTOR_SIZE))

# Generate a list of equally spaced points between the two endpoints
STEPS = 10
latent_space_samples = torch.empty(STEPS, LATENT_VECTOR_SIZE)
for i in range(STEPS):
    latent_space_samples[i] = torch.lerp(latent_space_endpoints[0], latent_space_endpoints[1],float(i)/float(STEPS+10))

# Generate images from the list of points
generated_samples = generator(latent_space_samples)
generated_samples = generated_samples.detach()

# Display the images in order
fig, axes = plt.subplots(1, STEPS, figsize=(10,2))
fig.text(0.5, 0.8, "Looking at the lerping betweent two flowers", ha='center', va='center')

for i  in range(STEPS):
    sub = plt.subplot(1, STEPS, i+1)
    sub.axis('off')
    image = generated_samples[i].permute(1,2,0)
    axes[i].imshow(image * 0.5 + 0.5)
plt.tight_layout(pad=0.5)
fig.show()

## Stats

In [ ]:
plot = plt.figure()
plt.plot(range(0, num_epochs), timeline["loss_discriminator"], label="Discriminator loss")
plt.plot(range(0, num_epochs), timeline["loss_generator"], label="Generator loss")
plt.title("Loss")
plt.legend()
plot.show()

# Save model

It's a dumb and simple saving system. To save the generator, set `SAVE` to true, and give it a `NAME`. To load a previously saved generator, set `SAVE` to false, and input the `NAME` of the model to load.

In [ ]:
SAVE = False
NAME = "Flowers300"
path = f"saved/{NAME}.gangrud"
if SAVE:
    torch.save(generator.state_dict(), path)

## View saved model

In [ ]:
saved_model = Generator()
saved_model.load_state_dict(torch.load(path))

In [ ]:
show_examples(saved_model, title="Saved Model", big=True)